In [ ]:
# Preperare distributed
using Distributed
addprocs(Sys.CPU_THREADS - nprocs() - 1)
print("Number of workers: ", nprocs(), "\nNumber of CPU threads: ", Sys.CPU_THREADS, "\n")

@everywhere using LoopVectorization
using BenchmarkTools

@everywhere using Plots
include("distributed_gif.jl")

using Base.Threads
print("Number of threads: ", Threads.nthreads(), "\n") # Check number of threads available

# Problem 1.2

In [ ]:
N = 100
L = 1.0
dx= L/N
D = 1.0
dt = 0.00001
c_0 = zeros(N, N)
c_0[:,end] .= 1.0
plot(heatmap(c_0', aspect_ratio=1, title="Initial condition", xlabel="x", ylabel="y"))

In [ ]:
"Enable benchmarks to test which version is faster. Set `do_bench = true` to run benchmarks."
do_bench = true

In [ ]:
function c_next(c::Matrix{Float64}, D::Float64, dx::Float64, dt::Float64)
    N = size(c, 1)
    c_new = similar(c)

    # Apply boundary conditions
    c_new[:, 1] .= 0.0  # Left boundary
    c_new[:, end] .= 1.0  # Right boundary

    Fo = D * dt / dx^2

    @inbounds @turbo for i in 2:N-1, j in 2:N-1
        c_new[i, j] = c[i, j] + Fo * (c[i+1, j] + c[i-1, j] + c[i, j+1] + c[i, j-1] - 4 * c[i, j])
    end

    @inbounds for j in 2:N-1
        c_new[1, j] = c[1, j] + Fo * (c[2, j] + c[N, j] + c[1, j+1] + c[1, j-1] - 4 * c[1, j])
        c_new[N, j] = c[N, j] + Fo * (c[1, j] + c[N-1, j] + c[N, j+1] + c[N, j-1] - 4 * c[N, j])
    end

    return c_new
end
if do_bench
    @benchmark c_next($c_0, $D, $dx, $dt)
end

In [ ]:
function c_next_single_loop(c::Matrix{Float64}, D::Float64, dx::Float64, dt::Float64)
    N = size(c, 1)
    c_new = zeros(N, N)
    @inbounds @turbo for i in 1:N, j in 2:N-1
        # Project x-end to x-start (periodic boundary condition)
        i_right = i == N ? 2 : i + 1
        i_left = i == 1 ? N - 1 : i - 1

        c_new[i, j] = c[i, j] + (D * dt / dx^2) * (c[i_right, j] + c[i_left, j] + c[i, j+1] + c[i, j-1] - 4 * c[i, j])
    end

    # Apply boundary conditions
    c_new[:, 1] .= 0.0  # Left boundary
    c_new[:, end] .= 1.0  # Right boundary
    return c_new
end
if do_bench
    @benchmark c_next_single_loop($c_0, $D, $dx, $dt)
end

In [ ]:
function c_next_dist_turbo(c::Matrix{Float64}, D::Float64, dx::Float64, dt::Float64)
    N = size(c, 1)
    c_new = similar(c)

    # Apply boundary conditions
    c_new[:, 1] .= 0.0  # Left boundary
    c_new[:, end] .= 1.0  # Right boundary

    Fo = D * dt / dx^2

    @inbounds @distributed for i in 2:N-1
        @inbounds @turbo for j in 2:N-1
            c_new[i, j] = c[i, j] + Fo * (c[i+1, j] + c[i-1, j] + c[i, j+1] + c[i, j-1] - 4 * c[i, j])
        end
    end

    @inbounds @turbo for j in 2:N-1
        c_new[1, j] = c[1, j] + Fo * (c[2, j] + c[N, j] + c[1, j+1] + c[1, j-1] - 4 * c[1, j])
        c_new[N, j] = c[N, j] + Fo * (c[1, j] + c[N-1, j] + c[N, j+1] + c[N, j-1] - 4 * c[N, j])
    end

    return c_new
end
if do_bench
    @benchmark c_next_dist_turbo($c_0, $D, $dx, $dt)
end

In [ ]:
function c_next_dist(c::Matrix{Float64}, D::Float64, dx::Float64, dt::Float64)
    N = size(c, 1)
    c_new = similar(c)

    # Apply boundary conditions
    c_new[:, 1] .= 0.0  # Left boundary
    c_new[:, end] .= 1.0  # Right boundary

    Fo = D * dt / dx^2

    @inbounds @distributed for z in 1:(N-2)*(N-2)
        i = mod(div(z-1, N-2), N-2) + 2
        j = mod(z-1, N-2) + 2
        c_new[i, j] = c[i, j] + Fo * (c[i+1, j] + c[i-1, j] + c[i, j+1] + c[i, j-1] - 4 * c[i, j])
    end

    @inbounds @turbo for j in 2:N-1
        c_new[1, j] = c[1, j] + Fo * (c[2, j] + c[N, j] + c[1, j+1] + c[1, j-1] - 4 * c[1, j])
        c_new[N, j] = c[N, j] + Fo * (c[1, j] + c[N-1, j] + c[N, j+1] + c[N, j-1] - 4 * c[N, j])
    end

    return c_new
end
if do_bench
    @benchmark c_next_dist($c_0, $D, $dx, $dt)
end

In [ ]:
function propagate_c_diffusion(c::Matrix, L::Float64, N::Int, D::Float64, t_0::Float64, t_f::Float64, dt::Float64)
    c_curr = copy(c)
    dx = L / N

    # Check stability condition
    if D * dt / dx^2 > 0.25
        error("Stability condition violated: D*dt/dx^2 must be <= 0.25")
    end

    for t in t_0:dt:t_f
        c_curr = c_next(c_curr, D, dx, dt)
    end

    return c_curr
end

In [ ]:
c_0001 = propagate_c_diffusion(c_0, L, N, D, 0.0, 0.001, dt)
c_001 = propagate_c_diffusion(c_0001, L, N, D, 0.001, 0.01, dt)
c_01 = propagate_c_diffusion(c_001, L, N, D, 0.01, 0.1, dt)
c_1 = propagate_c_diffusion(c_01, L, N, D, 0.1, 1.0, dt)

p_yrofile = plot(c_0001[1,:], title="y-profile", xlabel="x", ylabel="y", label = "t = 0.001")
plot!(c_001[1,:], label = "t = 0.01")
plot!(c_01[1,:], label = "t = 0.1")
plot!(c_1[1,:], label = "t = 1.0")

# Plot heatmaps at these time points
p_heatmap_0001 = plot(heatmap(c_0001', aspect_ratio=1, title="t = 0.001", xlabel="x", ylabel="y"))
p_heatmap_001 = plot(heatmap(c_001', aspect_ratio=1, title="t = 0.01", xlabel="x", ylabel="y"))
p_heatmap_01 = plot(heatmap(c_01', aspect_ratio=1, title="t = 0.1", xlabel="x", ylabel="y"))
p_heatmap_1 = plot(heatmap(c_1', aspect_ratio=1, title="t = 1.0", xlabel="x", ylabel="y"))

plot(p_yrofile, p_heatmap_0001, p_heatmap_001, p_heatmap_01, p_heatmap_1, layout=(5,1), size=(800,1200))